# Cayley-Hamilton Demo

<https://en.wikipedia.org/wiki/Cayley%E2%80%93Hamilton_theorem>


## Parameters


In [1]:
# from pprint import pprint

In [ ]:
VERBOSITY = 100
set_random_seed(31337)

q, n = (
    7,
    4,
    # 19, 8
    # 997, 16
    # 3329, 256
)

## Setup


In [ ]:
assert isinstance(q, Integer), "q is not an Integer!"
assert isinstance(n, Integer), "n is not an Integer!"

assert q >= n, "If n >= q, then n = q + e, and a^n = a^(q+e) = a^(1+e) - loops before!"

Fq = GF(q)
PRq.<x> = Fq['x']

print(f"""
Fields setup up! (Params: {q=}, {n=})

{Fq=}
{PRq=}
""")

In [ ]:
Fq_elems = list(Fq)
if VERBOSITY >= 1:
    print(f"{Fq_elems=}")

## Random Selection


$$
\mathcal{J}_\lambda(A) := \{ J_{\lambda,j}(A) : 1 \le j \le s_\lambda \}
$$

$$
\mathcal{J}(A) := \bigcup_{\lambda \in \Lambda(A)} \mathcal{J}_\lambda(A)
$$


#### Recap - Linear Algebra

##### Jordan chain

- Jordan chain ($J_{\lambda,j}(A)$): The $j$-th chain of generalised eigenvectors ($v_1, v_2, \dots, v_k$) in $A$, whose shared eigenvalue is $\lambda$ and size $k$. - **Key definition**: $$
  \fbox{%
  \(
  v_i =
  \begin{cases}
  \textbf{ordinary: } & (A - \lambda I)v_1 = 0, & i = 1,\\[1mm]
  \text{non-ordinary: } & (A - \lambda I)v_i = v_{i-1}, & 2 \le i \le k.
  \end{cases}\)%}
  $$
      - Thus, $(A - \lambda I)^i v_i = 0$
      - And intuitively:
          - $\exists v_1 \overset{A -\lambda I}{\mapsto} 0$
          - $\exists v_2 \overset{A -\lambda I}{\mapsto} v_1$
          - $\dots$
          - $\exists v_k \overset{A -\lambda I}{\mapsto} v_{k-1}$
          - $\not \exists v_{k+1} \overset{A -\lambda I}{\mapsto} v_k$
  $$

##### Jordan Chain Components

- A matrix $A \in \mathbf{K}^{n \times n}$ can be characterised as having $n$ generalised eigenvectors, distributed among:

  - $\Lambda(A)$: Set of distinct generalised+ordinary eigenvalues in $A$
  - $G_\lambda(A) := \ker\big((A-\lambda I)^n\big)$: Generalised eigenspace of shared eigenvalue $\lambda$
    - Generalised eigenspace: $\mathcal{G}(A) := \bigoplus_{\lambda \in \Lambda(A)} G_\lambda(A)$
    - $\mathcal{G}(A) \equiv K^n$: i.e. generalised eigenspace spans the basis space (connects to always $n$ generalised basis eigenvectors)
  - $E_\lambda(A) := \ker(A-\lambda I)$: Ordinary eigenspace of shared eigenvalue $\lambda \neq 0$
    - Ordinary Eigenspace: $\mathcal{E}(A) := \bigcup_{\lambda \in \Lambda(A) / 0} E_\lambda(A)$
    - $\mathcal{E}(A) \subseteq \mathcal{G}(A)$: The generalised eigenspace encompasses the ordinary eigenspace (since it's just the first eigenvector of each Jordan chain)

- Now, these generalised eigenspaces of specific eigenvalue $\lambda$ can be further partitioned by which Jordan chains form that eigenspace
  - $\mathcal{J}_\lambda(A) := \{ J_{\lambda,j}(A) : 1 \le j \le s_\lambda \}$: All Jordan chains of eigenvalue $\lambda$ in $A$ (of which there are $s_\lambda$ distinct chains)
    - $\mathcal{J}(A) := \bigcup_{\lambda \in \Lambda(A)} \mathcal{J}_\lambda(A)$: All Jordan chains in $A$
  - Thus, the generalised eigenspaces of $\lambda$ can be decomposed by jordan chains of eigenvalue $\lambda$ per:
    - $G_\lambda(A) = \bigoplus \mathcal{J}_\lambda(A) = \operatorname{span} \Big( \bigcup \mathcal{J}_\lambda(A) \Big)$
    - $G(A) = \bigoplus \mathcal{J}(A) = \operatorname{span} \Big( \bigcup \mathcal{J}(A) \Big)$

Thus, the overall decomposition of a matrix is:

- $n$ distinct generalised eigenvectors, spanning $\mathcal{G}(A) \equiv K^n$
- Partitioned into $\|\Lambda(A)\|$ subspaces of $G_\lambda(A)$, according to shared eigenvalues
- Further partitioned into $s_\lambda$ jordan chains of $J_{\lambda,j}(A)$, according to shared generalised eigenvector power series/chain
- And lastly partitioned into $s_\lambda$ ordinary eigenvectors at the tip of each jordan chain, and $\dim G_\lambda(A) - s_\lambda$ non-ordinary eigenvectors

##### Characteristic/Minimal polynoials

- Characteristc polynomial: $$
  \boxed{~\chi*A(x) \coloneqq \det(xI - A) \equiv \prod*{J*{\lambda,i} \in \mathcal{J}(A)} (x - \lambda)^{\operatorname{Size}(J*{\lambda,i}(A))}~}

  $$
      - i.e. it's the *polynomial product of **all jordan chains**, indexed by their eigenvalues $\lambda$ as roots and size (computed via $\operatorname{Size}(\cdot)$) as exponents*
      - Note that if distinct jordan chains have overlapping eigenvalues $lambda_i$, for $\chi_A$ they constructively add
  $$

- Minimal polynomial: $$
  \boxed{~m*A(x) \coloneqq (\min*{K[x]}{m*A(A)=0}) \equiv \prod*{\lambda \in \Lambda(A)} (x - \lambda)^{\max*{i \in s*\lambda}(\operatorname{Size}(J\_{\lambda,i}(A)))}~}
  $$
      - i.e. it's the *polynomial product of **dominating jordan chains**, indexed by their eigenvalues $\lambda$ as roots and largest size (computed via $\max_{i \in s_\lambda}(\operatorname{Size}(\cdot))$) as exponents*
      - Note that if distinct jordan chains have overlapping eigenvalues $\lambda_i$, for $m_A$ they destructively compete, and only the largest chain is expressed
  $$

##### Matrix classification via polynomial properties

Now, the simple sub-cases you can consider, by just asking 'when is this partitioning layer irrelevant':

- What if, in each jordan chain, there was just one vector? (i.e. all jordan chains are of length 1)

  - $m_A(x)$ has no overlapping factors ($\max(1,1 \dots 1) = 1$)
  - Formally:
    - Jordan chains = 1x1 scalar matrices
      - Jordan normal form will just be a diagonal
    - Also, there are only Ordinary eigenvectors; no non-ordinary eigenvectors
      - Thus, $\mathcal{E}(A) = \mathcal{G}(A) \equiv K^n$
      - Ergo, $\mathcal{E}(A) \equiv K^n$
      - i.e. Ordinary eigenvectors span the whole of $K^n$! It's a full basis!
    - A.k.a; matrix is diagonalisable!

- What if, in each shared eigenvalue partition of eigenspaces, there is only one jordan chain?
  - $\chi_A(x) = m_A(x)$ ($\max(a) = a$)
  - Formally: - no overlapping chains means all jordan chains == all dominating jordan chains) - geometric multiplicity = algebraic multiplicity
    - This is called a 'cyclic matrix'!
      - Formally, it's because its power series is itself a generator for $\mathbb{K}[x]$
      - Informally; it means it neatly fits into the $\mathbb{K}[x]/g(x)$ paradigm we will explore later, since its generator qualities (outlined by the minimal polynomial) lets it fully explore all its associated characteristics (outlined by the characteristic polynomial)


### Quotient


#### Disclaimer

Funky stuff happens in $\mathbb{K}[x]/g(x)$ when the quotient $g(x)$ isn't fully factorizable into unique root in our coefficient field $\mathbb{K}$

1. Since $\mathbb{K}[x]$ is a PID, when we get $\mathbb{K}[x]/g(x)$, we can decompose it via CRT; but only into the prime ideals $\mathbb{K}[x]$ can support expressing

   - This is because in a PID, $g(x) = \prod_{i=1}^r p_i(x)^{e_i} : p_i(x) \in \mathrm{Irr}(\mathbb{K}[x])$
   - And often, these are our linear prime ideals of $(x - a)$; which is perfect, since $\forall a \in \mathbb{K} : \mathbb{K}[x]/(x - a) \cong \mathbb{K}[x]$

     - This is as $(x - a) = 0 \iff x = a$, so your polynomials just trivially resolve to a constant value in $\mathbb{K}$

   - Moreover, if $\mathbb{K}[x]$ can always be factored into order-$n$ roots, we would always be able to decompose $g(x)$ fully into linear prime ideals of $p_i(x)^{e_i} = (x - a)^{e_i}$

     - More formally: $\mathbb{K}$ is an algebraicly closed field like ($\mathbb{C}$), if in contains the roots of all polynomials you can express with it
     - LINK: This guarantee's connection to linear algebra, is that whilst a matrix $A$ will always have $n$ _generalized_ eigenvectors, its generalized eigenvalues are the roots of $\chi_A(x)$ ($A$'s characteristic polynomial); but can these roots be expressed in $\mathbb{K}$?
       - Note that generalized eigenvectors != ordinary eigenvectors (<https://en.wikipedia.org/wiki/Generalized_eigenvector>)
         - The former are the general, foundational principle of known quantity (always $n$ for $n \times n$ matrix), which we use to build the latter of variable grouping
         - Aka 'ordinary eigenvector' := 'useful nonzero family chain of generalized eigenvectors'; see recap for details
       - And, in an algebraically complete field, the characteristic polynomial always fully factors into $n$ roots, so its $n$ generalised eigenvectors/eigenvalues can all be expressed

   - However, if we are in an algebraically incomplete field, like $\mathbb{Q}$, $\mathbb{R}$, or $\mathbb{F}_p$, this lack of expressibility can yield problems...
     - i.e. $\exists p^\prime(x) \in \mathrm{Irr}(\mathbb{K}[x])$ that is not linear, yet is irreducible!
     - This is as our coefficient field $\mathbb{K}$ has hit its algebraic limits! (should have worked in $\overline{\mathbb{K}}$ 😏)
       - Now, if your quotient is irreducible, then:
         - The resulting quotient ring is a field (called a splitting field)
         - _And_ can express all statements originally in $\mathbb{K}$; i.e. it's a field extension
         - But even more importantly; _the splitting field of $\mathbb{K}[x]/p^\prime(x)$ is equivalent to extending $\mathbb{K}$ by the roots of $p^\prime(x)$_
           - i.e. $$\boxed{\mathbb{K}(\alpha_1 \dots \alpha_n) \cong \mathbb{K}[x]/p^\prime(x) \quad \forall i : p^\prime(\alpha_i) = 0}$$
         - Thus, you can update your coefficint field with this ring, and irreducible polynomial's roots could be used to extend $\mathbb{K}$ into its splitting field
       - More formally: $\overline{\mathbb{K}}$ is the algebraic closure of $\mathbb{K}$, constructed by extending $\mathbb{K}$ with all its missing polynomial roots until it has become algebraically closed
       - E.g. $\mathbb{R}(\alpha) := \mathbb{R}[x]/(x^2 + 1)$, where $\alpha : \alpha^2 + 1 = 0 \iff \alpha^2 = -1 \iff \alpha = \sqrt{-1}$
         - Aka $\mathbb{C} := \mathbb{R}(i) := \mathbb{R}[x]/(x^2 + 1)$
       - And critically, **if $\mathbb{K} \cong \overline{\mathbb{K}}$, then all $\mathbb{K}[x]/g(x)$ can be decomposed into a problem in \mathbb{K}**
         - _This is also why 'the fundamental theorem of algebra'; that $\mathbb{C}$ factors all polynomials, is so important_
           It means that $\mathbb{C}$ is the apex of this type of field extension on $\mathbb{Q}$, and in turn, _$\overline{\mathbb{K}}$ (e.g. $\mathbb{C}$), can always express its roots and eigenvalues_
       - However, it must be noted that whilst $\overline{\mathbb{K}}$ can always express problems in $\mathbb{K}$, $\overline{\mathbb{K}}$ sometimes does lose useful properties that existed in $\mathbb{K}$!
         - For example, $\mathbb{R}$ is totally ordered; but $\mathbb{C}$ is not!
     - LINK: This dilemma's equivalent in linear algebra is that when our minimal polynomial in $\mathbb{K}$ is trying to realise an eigenvalue _outside of $\mathbb{K}$_, you cannot jordan-diagonalize that matrix with coefficients in $\mathbb{K}$; you have to use $\overline{\mathbb{K}}$
       - This is one possible (albeit rare) reason why, even though there are always $n$ _general_ eigenvectors, you end up with fewer proper eigenvectors

---

2. Thus, we can decompose this ring into smaller subrings
   - i.e. $\mathbb{K}[x]/f(x) \cong \prod_{i=1}^r \mathbb{K}[x]/(p_i(x)^{e_i})$
     - This decomposition's equivalent in linear algebra is Jordan Diagonalization
     - There, you describe generalised eigenvectors, which get grouped under shared eigenvalues (when this is a stand-alone group, it's a conventional eigenvector)
   - There, each _linear_ $p_i(x)$ becomes a Jordan block, and that block's size is $e_i$

- As such, we will build our quotient from unique order-1 roots directly!
  - This means our quotient is always fully reducible; no nasty irreducible factors
  - And it means our quotient

Also, our quotient's roots must be unique!

- This is as its equivalent to our eigenvalues overlapping


In [ ]:
g_roots = sample(Fq_elems, n)
g_factors = [(x - root) for root in g_roots]
g = prod(g_factors)

print(
    f"""
Random reduction polynomial g(x) selected!
{g=}
{g_roots=}
"""
)

In [ ]:
PRq_gx.<G> = PRq.quo(g)
PRq_gx_basis = dict(enumerate(G.powers(n)))
G_Cg=G.matrix()

print(f"""
{PRq_gx=}
G_Cg=G.matrix()=\n{G_Cg}

Note that the bottom-row of our matrix is encoding the quotient-polynomials non-monic coefficients, albeit negated
{(x^n - g)=}
""")

In [ ]:
print(
    "This yields us a basis over which our coefficients of f(x) in f(x)/g(x) will be evaluated over:"
)
if VERBOSITY >= 0:
    # for k, v in PRq_gx_basis.items(): print(f"G^{k} = \n{v.matrix()}\n")
    # pprint({f'G^{k}': v.matrix() for k, v in PRq_gx_basis.items()})
    display(
        table(
            [
                [name] + list(map(func, PRq_gx_basis))
                for name, func in (
                    {
                        "id": lambda k: k,
                        "val": lambda k: PRq_gx_basis[k],
                    }
                    | {
                        f"row_{i}": lambda k, i=i: (PRq_gx_basis[k]).matrix()[i]
                        for i in range(n)
                    }
                ).items()
            ]
        )
    )

### Element


In [ ]:
r_poly = PRq.random_element(degree=n - 1, monic=false)
r = PRq_gx(r_poly)

print(
    f"""
Random test polynomial:
{r=}

And it has a Matrix form:
{r.matrix()}

Whose top row is:
{r.matrix().row(0)}
"""
)

In [ ]:
print("v's matrix form is just G.matrix(), evaluated under r_poly")
assert r.matrix() == r_poly(G_Cg), "mismatch in matrices! IDK what could cause this!"
print(
    "And you can clearly read r_poly from v, as its first row is the reversed r_poly coeffs"
)
assert list(r.matrix().row(0)) == list(
    r_poly.coefficients(sparse=False)
), "mismatch in coefficients! Is it another sparseness/type error?"

if VERBOSITY >= 2:
    print(
        "You can also very clearly see that this matrix can be constructed by treating G^n as your basis, and the polynomial coefficients as your vector values!"
    )
    r_total = 0
    for coeff in zip(r_poly.coefficients(sparse=False), PRq_gx_basis.values()):
        r_coeff_mat = prod(coeff).matrix()
        r_total += r_coeff_mat
        print(
            f"""{coeff[0]}*{coeff[1]}=
{r_coeff_mat}
coefficient total =
{r_total}
"""
        )
    assert (
        r.matrix() == v_total
    ), "Something went wrong in my coefficient-by-coefficient aggregation!"

print("All held!")

## Proof By Exhaustion


In [ ]:
from dataclasses import dataclass


@dataclass
class PolyDiag:
    poly: type(G)
    mat: type(G_Cg)
    diag: type(G_Cg)

In [ ]:
# [Jordan Algebras - Algebras](https://doc.sagemath.org/html/en/reference/algebras/sage/algebras/jordan_algebra.html)
# [p.matrix().jordan_form(
# eigenvalues=g_roots
# transformation=True
# ) for p in PRq_gx]
# or .diagonalization()
# HOWEVER - this does not standardize their diagonaliations! Why? Because they each yield the same eigenvectors, but use a different permutation of eigenvectors!
# As such, need to do this the other way round! Get a standard diagonalization and transform, and show that this transform matrix is universal!

assert (
    G_Cg.is_diagonalizable()
), "Companion Matrix of G *should* be diagonalizable!!! WTF! Perhaps we have a repeated root?"
G_Cg_diag, G_diagE = G_Cg.diagonalization()
assert (
    G_Cg == G_diagE * G_Cg_diag * ~G_diagE
), "Diagonalization reconstruction didn't work!"
assert G_Cg_diag == ~G_diagE * G_Cg * G_diagE, "Manual Diagonalization didn't work!"


def manual_diag(poly_m, eigen_m, verbose=False):
    assert poly_m.is_diagonalizable(), "Your matrix should be diagonalizable!"

    poly_m_diag = ~eigen_m * poly_m * eigen_m
    # if verbose: print(f"poly_m=\n{poly_m}\npoly_m_diag=\n{poly_m_diag}\n")

    assert (
        poly_m == eigen_m * poly_m_diag * ~eigen_m
    ), f"Inversion with this eigenbasis isn't happening! {poly_m=}\n{poly_m_diag=}\n{eigen_m=}"
    assert (
        poly_m_diag.is_diagonal()
    ), f"Diagonalization with this eigenbasis failed! {poly_m_diag=}"

    return poly_m_diag


def map_and_assert_poly(poly, eigen_m=G_diagE, verbose=True):
    poly_m = poly.matrix()
    if verbose:
        print(
            f"""
{poly=}
poly_m=
{poly_m}
"""
        )

    poly_diag = manual_diag(poly_m, eigen_m, verbose=verbose)
    if verbose:
        print(
            f"""
poly_diag=
{poly_diag}
"eigen_m=
{eigen_m}
"""
        )

    return (poly, poly_m, poly_diag)


PRq_gx_diags = list(map(map_and_assert_poly, PRq_gx))

In [ ]:
# CHECK IS NOT WORKING FOR G^2!

# .is_permutation_of?
# .with_permuted_rows_and_columns?


def map_and_assert_poly(poly, eigen_m=G_diagE, verbose=True, check=True):
    poly_m = poly.matrix()
    if verbose:
        print(f"{poly=}")
        print(f"poly_m=\n{poly_m}\n")

    poly_diag = manual_diag(poly_m, eigen_m, verbose=verbose)
    if verbose:
        print(f"poly_diag=\n{poly_diag}")
        print(f"eigen_m=\n{eigen_m}\n")
    # GIVING UP BECAUSE IF THE POLYNOMIAL's MATRIX HAS A large charpoly() / minpoly(), it can use a simpler eigenbasis, since whilst it will follow the simultaineous eigenbasis, it uses several pairs of eigenvectors with the same eigenvalue. Ergo, it can be represented as a simple eigenbasis
    # Also, just realised - if g(x) can be expressed as g'(x^n), ofc stuff simplifies for G^2... Great. Had q=7, g=(x^4 + 4*x^2 + 2)
    # TLDR - if f(x)/g(x) has repeated eigenvalues, you can mix and match the eigenvectors across this overlap, and the libraries often simplify it this way.
    if check:
        if verbose:
            print("Checking...")

        poly_diag_auto, eigen_m_auto = poly_m.diagonalization()
        is_diag_match, diag_perm = poly_diag_auto.is_permutation_of(
            poly_diag, check=True
        )
        is_eigen_match, eigen_perm = eigen_m_auto.is_permutation_of(eigen_m, check=True)

        if verbose:
            print(f"{poly_diag_auto=}\n{eigen_m_auto=}\n")
            print(f"{is_diag_match=}\n{diag_perm=}\n{is_eigen_match=}\n{eigen_perm=}\n")

        assert (
            is_diag_match
        ), "The manual and automatic calculations for the diagonals don't match!"
        assert (
            diag_perm[0] == diag_perm[1]
        ), "Diagonal permutations should be symmetric!"

        if (
            not eigen_m_auto.is_one()
        ):  # For multiples of the identity matrix, this is always 1, not the true eigenvectors
            assert (
                is_eigen_match
            ), "The manual and automatic calculations for the eigenbasis don't match!"
            assert (
                eigen_perm[0].absolute_length() == 0
            ), "The eigenbasis's actual basis vectors should be unchanged!"
            assert (
                diag_perm[1] == eigen_perm[1]
            ), "The diagonal and eigenbasis should be shuffled around the same way!"

    return (poly, poly_m, poly_diag)

- First PHd (Dennis?) student wrote a security proof for the PAKE protocol
  - Increased the security definition, and went one step further with security against pre-calculated attacks
- READ THE PAPERS LISTED

Andre 4 admin (do they have a local binder service?)


In [ ]:
k.<a> = GF(2^2000)
k_ord = k.zeta_order()

print(k_ord)

#k.zeta(p_given)
p_given = 8877945148742945001146041439025147034098690503591013177336356694416517527310181938001
facts = Factorization([(p_given, 1)], cr=True, sort=True, simplify=True)

# par_facts = k_ord.factor(limit=10^9)
# small_facts, rem_facts = par_facts[:-1], par_facts[-1]
# small_facts
facts *= Factorization([(3, 1), (5, 4), (11, 1), (17, 1), (31, 1), (41, 1), (101, 1), (251, 1), (257, 1), (401, 1), (601, 1), (1601, 1), (1801, 1), (4001, 1), (4051, 1), (7001, 1), (8101, 1), (25601, 1), (28001, 1), (61681, 1), (76001, 1), (96001, 1), (268501, 1), (340801, 1), (1074001, 1), (2020001, 1), (2787601, 1), (3775501, 1), (22624001, 1), (42144001, 1), (82471201, 1), (229668251, 1)])

#

# # For the remainder, had to use <https://www.factordb.com/>
# db_facts
facts *= Factorization([(3173389601, 1), (269089806001, 1)])

# # NOTE: LEARNED THE HARD WAY THAT FACTORDB DOESN'T CLEARLY DIFFER BETWEEN PRIMES AND FACTORS!
# # In this case - `1860077361115047584755563693343361`
# db_pari_facts
facts *= Factorization([(4278255361, 1), (293543676001, 1), (1481124532001, 1)])

# Found by running `factor(rem_facts, proof=True, algorithm="pari", verbose=8)`, and waiting for `found factor = ...`
# pari_facts - ``
facts *= Factorization([(94291866932171243501, 1), (4710883168879506001, 1), (47970133603445383501, 1)])

for p, n in facts:
    assert p.is_prime(proof=True), f"{p, n=} is not a prime power!"
facts_n = facts.value()

assert facts_n.divides(k_ord), "pseudo-factorization isn't actually part of k_ord!"
cofact_n, rem = divmod(k_ord, facts_n)
assert rem == 0, "divmod wasn't clean - bro wtf, should have been caught earlier"
facts, cofact_n

if cofact_n.is_prime(proof=True):
    print("Primes found!")
else:
    print("Factoring...")
    factor(cofact_n, proof=True, algorithm="pari", verbose=4)

#  94291866932171243501 4710883168879506001, 47970133603445383501

# Scratchwork


## Kyber Scratchwork


In [ ]:
q = 3329
n = 256
Rq = GF(q)
PRq.<x> = PolynomialRing(GF(q))
PRq_cyclic.<zc> = PRq.quotient(x^n - 1)
PRq_ncyclic.<zn> = PRq.quotient(x^n + 1)

fact_cyclic = PRq_cyclic.modulus().factor()
fact_cyclic
len(fact_cyclic)

fact_ncyclic = PRq_ncyclic.modulus().factor()
fact_ncyclic
len(fact_ncyclic)

# from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
# cyclotomic_coeffs(512)
PR_cyclotomic = CyclotomicField(512)
PR_cyclotomic.absolute_polynomial()
# .absolute_polynomial_ntl() = absolute_polynomial, but scaled up from Q to integers - guarantees scaling

# TODO: figure out `Rq.cyclotomic_cosets()` - this is likely the secret

# ALSO: looks fascinating!
# Rqs = GF(q^2)
## Can pass it as a tuple to avoid factoring! (q, 2)
# Rqs.factored_order()
# Rqs.degree()
# Rqs.vector_space()
# Rqs.subfields()
# Rqs.frobenius_endomorphism
# ??? subsemigroup()
# ??? rhodes_radical_congruence()
# Takes ages - .factored_unit_order()
# Takes ages - .idempotents()

# ALSO!!!! Rqs.polynomial_ring()
# And look into .quotient_by_principal_ideal(), and .quo()

# GF(q).polynomial_ring().quo(x^256 - 1, 'x').random_element().fcp()
# Also, what about .charpoly('x')?

# TODO: LOOK INTO .matrix() and .minpoly()!!!
# Also, try to apply minpoly beforehand for random_element
# Actually, `fcp` doesn't exist for `minpoly`
# HOWEVER - .minpoly().factor() seems to be .fcp(), but w/o repeated factors!
# It may be necessary to distinguish the 'x' from the polynomial ring, and those from the quotient ring

In [ ]:
q = 3329
n = 256

Rq = GF(q)
PRq.<x> = Rq[]
PRq_cyclic.<z> = PRq.quo(x^n - 1)

v = PRq_cyclic.random_element()
display(
    v.fcp()
    v.minpoly().factor()
    z.matrix().str()
)

# What does .pseudo_order() do?
# And .field_extension()?
# And .echelon_form()?

# I think .fraction_field() means polynomials which can be fractions of each other.

# And what about this syntax:
# K.<k> = GF((q,2))['k']
# Think it's shorthand for PolynomialRing(..., 'k') or .polynomial_ring()
# So write PRq.<x> = GF((q, 256))['x']
# Except this immediately tries to create an optimal modulus
# Skip by not specifying field size, and immediately getting your quotient size
# So use `GF(q)['x'].quo(x^n - 1)`
# Or use '.extension()'? Apparently this is specific to fields, whilst quotient is more general - but lets try using it for now
# So use B.<x> = GF(q)['x'].extension(x^n - 1)

# B.coerce_embedding
# B.coerce_map_from
# B.cover

# Actually, it looks soo wordy with extension - for now, stick with quo
B.<x> = GF(q)['x'].quo(x^n - 1)

# Also, look into B.hom()! May be connected to the codomain

# MIGHT BE - GF() can take a `modulus` argument?

# Also, 'CyclotomicField'
# `.semigroup_generators`
# `.zeta` `.unit`


# WTF is `modular_composition` and `minpoly_mod`

In [ ]:
q = 3329
n = 256

Rq = GF(q)
PRq.<x> = Rq[]
PRq_cyclic.<z> = PRq.quo(x^n - 1)

r_poly = PRq.random_element(degree=255)
v = PRq_cyclic(r_poly)
#v.fcp()
#v.minpoly().factor()
set(v.fcp())
{-r_poly(17^i) for i in range(256)}

# z.matrix().str()

What is `enumerate_totallyreal_fields_all`


In [ ]:
# m.cholesky
# # m.decomposition()